# TME 4 : Projet Detection de motifs

Nous allons développer des algorithmes pour chercher de motifs dans les données de ChipSeq de C. glabrata.
Pour commencer nous allons d'abord produire des données artificielles qui nous permettront de tester rapidement nos algorithmes. Ensuite nous allons chercher les motifs dans C. glabrata et analyser les résultats.

## Partie A : Recherche de pattern (motifs) identiques


1\. Faites une fonction pour générer aléatoirement des séquences artificielles, puis générer t=10 séquences de n=41 nucléotides chacune. Toutes les lettres peuvent être équiprobables pour la génération des séquences.

In [1]:
import random

nuc = ('A', 'C', 'G', 'T')

t=10 #nombre de sequences
n=41 #longueur des sequences

#Pour un simple teste
# t=3 #nombre de sequences
# n=8 #longueur des sequences

def generateRandomSequence(n, upper=False):
    """
    Genere une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    nucList = [nuc[random.randint(0,3)].upper() if upper else nuc[random.randint(0,3)].lower() for i in range(n)]
    sequence = "".join(nucList)
   
    return sequence

# print(generateRandomSequence(n))
# print(generateRandomSequence(n, True))

def generateRandomSequences(n, t):
    """
    Genere plusieurs séquences nucléotidiques aléatoires 
    entrée n : longueur des sequences
    entrée t : nombre de sequences
    sortie sequences : liste de sequences nucléotidiques aléatoires 
    """
    sequences = [generateRandomSequence(n) for i in range(t)]
   
    return sequences
    
seqs = generateRandomSequences(n, t);

print (seqs)


['ttggtcgtttttccgatgacagggacgtggcggtgttaggg', 'cccgatcccacaccggcccgttatttgattttttatggccc', 'tcaaaagtcatgacgggcgggtccgtaacgggtcataggtc', 'tcagtccagacacgattgtagcagacgccgccttatcgctc', 'tacagctgtttcccagacgtttgatatccgtttatggtggg', 'gagagattctggcgacctacttgaatagactcatcacattg', 'ttctacagccaagaggggtgtcccccactttagtctcctta', 'tttgcttcaagccttggagttcggtccaatggtctacagac', 'agatccgaacaccaggatggattcccatacatatgcggccc', 'ttgacgattcatgggggacaggtgttagtctctatctgggc']


2\. Nous allons maintenant implanter dans les séquences artificielles generés avant un motif de taille `k`=9 à des positions aléatoires (choisies uniformément le long de la séquence). On considère une proportion ``f``=0.9 de séquences qui possèdent le motif.

In [2]:
# taille des motifs k = 9
# k = 3 
k = 9
# frequences d'occurences des motifs f = 0.9 
f = 0.9 

import random

def implantMotifs(motifs, f, sequences):
    """
    Insérer un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée f : fréquence d'implantation
    entrée séquences : liste de sequences
    sortie modified_sequences: liste de séquences ayant parfois le motif implanté
    """
    length = len(sequences[0])
    assert([len(seq) for seq in sequences])

    upperFlag = sequences[0][0].isupper() # use the case of first nucleotide of sequences
    if(upperFlag):
        motifs = motifs.upper()
    else:
        motifs = motifs.lower()

    modified_sequences = []
    for seq in sequences:
        if random.random() <= f:
            pos_to_insert = random.randint(0, length)
            insertedSeq = seq[:pos_to_insert] + motifs + seq[pos_to_insert:]
            modified_sequences.append(insertedSeq)
        else:
            modified_sequences.append(seq)
    

    return modified_sequences

motifs = generateRandomSequence(k)
print("seqs, motif, motified")
print(seqs)
print(motifs)
motif_implanted_seqs = implantMotifs(motifs, f, seqs)
print(motif_implanted_seqs)


seqs, motif, motified
['ttggtcgtttttccgatgacagggacgtggcggtgttaggg', 'cccgatcccacaccggcccgttatttgattttttatggccc', 'tcaaaagtcatgacgggcgggtccgtaacgggtcataggtc', 'tcagtccagacacgattgtagcagacgccgccttatcgctc', 'tacagctgtttcccagacgtttgatatccgtttatggtggg', 'gagagattctggcgacctacttgaatagactcatcacattg', 'ttctacagccaagaggggtgtcccccactttagtctcctta', 'tttgcttcaagccttggagttcggtccaatggtctacagac', 'agatccgaacaccaggatggattcccatacatatgcggccc', 'ttgacgattcatgggggacaggtgttagtctctatctgggc']
gcggaccgt
['ttggtcgtttttccgatgacagggacgtggcggtgttaggg', 'cccgatcccacagcggaccgtccggcccgttatttgattttttatggccc', 'tcaaaagtcatgacggcggaccgtggcgggtccgtaacgggtcataggtc', 'tcagcggaccgtgtccagacacgattgtagcagacgccgccttatcgctc', 'tagcggaccgtcagctgtttcccagacgtttgatatccgtttatggtggg', 'gagaggcggaccgtattctggcgacctacttgaatagactcatcacattg', 'ttctacagccaagaggggtgtcccccagcggaccgtctttagtctcctta', 'tttgcttcaagccttggagtgcggaccgttcggtccaatggtctacagac', 'agatccgaacaccaggatggattccgcggaccgtcatacatatgcggccc', 'ttgacgattcatgggggcggaccgtgacaggtgttagt

3\. Faites une fonction pour chercher les $m$ motifs de taille $k$ les plus fréquents dans l'ensemble des séquences. Tester cette fonction sur un l'ensemble de séquences avec le motif implanté génère dans les questions précédentes. Faite aussi une fonction qu'affiche les $top$ motifs les plus fréquents. 

In [3]:
def sortDictMotifToList(motifDict : dict):
    return sorted(motifDict.items(), key=lambda item:item[1], reverse=True)

def searchMotifs(k, sequences):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    res = dict()
    totalValue = 0
    totalCount = 0
    for seq in sequences:
        if(len(seq)<k):
            continue
        seqRes = [seq[index:index+k] for index in range(len(seq)-(k-1))]
        for motif in seqRes:
            if motif in res.keys(): 
                res[motif] += 1
            else:
                res[motif] = 1
            totalValue += 1
        totalCount += len(seq)-(k-1)
        if(totalCount != totalValue):
            print(seqRes, totalCount, totalValue, len(seq)-(k-1), seq)
        assert(totalCount == totalValue)
    assert(sum(res.values()) >= sum([len(seq)-(k-1) for seq in sequences]))
    return res
# print(k, motif_implanted_seqs)
# print(searchMotifs(k, motif_implanted_seqs))

def getTopMotifs(motifs, top):
    """
    renvoyer les top motifs le plus frequent
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée top : les top plus frequent 
    sortie motifsfreq: dictionnaire contenant les top motifs les plus fréquents, clé=motif, valeur = fréquence d'observation
    >>>getTopMotifs({'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}, 2)
    {'TAA': 3, 'TAT': 2}
    """
    sortedDictMotif = sortDictMotifToList(motifs)
    motifsfreq  = {key_value[0]: key_value[1]  for key_value in sortedDictMotif[:top]}
    return motifsfreq



# motifsFound = searchMotifs(k, sequences)
motifsFound = searchMotifs(k, motif_implanted_seqs)
top = 5
topMotifs = getTopMotifs(motifsFound, top)
print(topMotifs)

{'gcggaccgt': 9, 'agcggaccg': 4, 'cggaccgtc': 4, 'cagcggacc': 3, 'ggcggaccg': 3}


4\. Quell est la complexité de votre algorithme?

<font color='blue'>
reponse : 
1. searchMotifs : <br/>
   A. Pour construire une liste intermédiaire des motifs, il faut faire l'indexage (len(seq)-(k-1)) fois, et à chaque fois on crée une copie de `seq` de taille k. Donc complexité pour cela est O(len(seq)-(k-1))= O(len(seq)-k)<br/>
   B. Pour mettre à jour le dictionnaire, on reparcourt la liste, ce qui fait aussi O(len(seq)-k)<br/>
   C. Au total, on parcourt tous les sequences. La somme de len(seq) fait O((n+k)*t) où n est la longueur de `sequences` générées, t le nombre de sequences générées et k la longueur de motif. La somme de `-k` fait `-t*k`. En ignorant le terme `-t*k`, a complexité totale fait <strong>O((n+k)*t)</strong>.<br/>
2. getTopMotifs : <br/>
   A. Il faut juste parcourt le dictionnnaire créé, donc fait aussi<strong>O((n+k)*t)</strong>.<br/>
</font>

5\. Certains motifs sont reverse complémentaires. Implanter des motifs reverse complémentaires dans les séquences déjà générés et en suite chercher ces motifs. Attention: vous devez réutiliser les fonctions développés précédemment, pas besoin de définir des nouvelles fonctions. 

In [4]:
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée : sequence de nucléotides (brin sens)
    sortie : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    res = ''
    for index in range(len(seq)):
        res += compl[seq[index].upper()]
    return res[::-1]

motifsReverseComplement = reversecompl(motifs)
reverse_motif_implanted_seqs = implantMotifs(motifsReverseComplement, f, seqs)
reverse_motif_found = searchMotifs(k, reverse_motif_implanted_seqs)
getTopMotifs(reverse_motif_found, top)

{'acggtccgc': 10,
 'gacggtccg': 4,
 'cggtccgct': 4,
 'ggacggtcc': 3,
 'cggtccgcc': 3}


## Partie B : Recherche de motifs identiques sur vos données

1\. Le fichier "C_glabrata_1000bp_only.fasta" contiens les séquences régulatrices pour tous les gènes de votre organisme. Nous avons pris les 1000bp en amont du codon start. Chercher les 50 motifs de taille 7 les plus fréquents dans les deux brim de votre génome.

In [5]:
k=7
feq=10
top= 50
genome = "Data/C_glabrata_1000bp_only.fasta"


def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    return sequence

sequence = readFasta(genome)


In [6]:
print("Searching motifs...\n")
motifsP2 = searchMotifs(k, sequence)

print("The most abundant motifs are \n")
topMotifsP2 = getTopMotifs(motifsP2, top)
topMotifsP2 = sorted(topMotifsP2.items(), key=lambda item:item[1], reverse=True)
print(topMotifsP2)

Searching motifs...

The most abundant motifs are 

[('AAAAAAA', 8385), ('TTTTTTT', 7658), ('ATATATA', 3096), ('ATTTTTT', 2970), ('AAAAAAT', 2960), ('TATATAT', 2796), ('AAAAATT', 2469), ('AATTTTT', 2457), ('GAAAAAA', 2417), ('AAAGAAA', 2417), ('AAAATTT', 2308), ('AAATTTT', 2305), ('TATTTTT', 2292), ('TTTTTTC', 2291), ('AAGAAAA', 2243), ('TATATAA', 2198), ('AAAAATA', 2188), ('TTTATTT', 2164), ('TTTCTTT', 2159), ('TGAAAAA', 2152), ('TTATTTT', 2134), ('TTTTATT', 2110), ('AAAAGAA', 2079), ('TTTTCTT', 2052), ('TTTTTCA', 1977), ('CAAAAAA', 1964), ('ATATAAA', 1946), ('AAAATAA', 1940), ('TTCTTTT', 1912), ('ATATATT', 1904), ('AATATAT', 1898), ('AATAAAA', 1889), ('AAATAAA', 1880), ('AGAAAAA', 1864), ('TTTTTAT', 1862), ('CTTTTTT', 1861), ('AAAAAAG', 1853), ('AATAATA', 1824), ('TTTTTTA', 1813), ('TTTTTTG', 1807), ('TTTTTCT', 1804), ('AAATATA', 1797), ('ATATTTT', 1781), ('ATAAAAA', 1780), ('TATATTT', 1771), ('TTATATA', 1734), ('TATTATT', 1729), ('AAAATAT', 1702), ('AAAAAGA', 1669), ('TAAAAAA', 1652

In [7]:
print("Searching reverse complementary motifs...\n")
reverseSequenceP2 = [reversecompl(seq) for seq in sequence]
motifsReverseP2 = searchMotifs(k, reverseSequenceP2)

print("The most abundant motifs found in reverse complementary are \n")
topReverseMotifsP2 = getTopMotifs(motifsReverseP2, top)
topReverseMotifsP2 = sorted(topReverseMotifsP2.items(), key=lambda item:item[1], reverse=True)
print(topReverseMotifsP2)

Searching reverse complementary motifs...

The most abundant motifs found in reverse complementary are 

[('TTTTTTT', 8385), ('AAAAAAA', 7658), ('TATATAT', 3096), ('AAAAAAT', 2970), ('ATTTTTT', 2960), ('ATATATA', 2796), ('AATTTTT', 2469), ('AAAAATT', 2457), ('TTTTTTC', 2417), ('TTTCTTT', 2417), ('AAATTTT', 2308), ('AAAATTT', 2305), ('AAAAATA', 2292), ('GAAAAAA', 2291), ('TTTTCTT', 2243), ('TTATATA', 2198), ('TATTTTT', 2188), ('AAATAAA', 2164), ('AAAGAAA', 2159), ('TTTTTCA', 2152), ('AAAATAA', 2134), ('AATAAAA', 2110), ('TTCTTTT', 2079), ('AAGAAAA', 2052), ('TGAAAAA', 1977), ('TTTTTTG', 1964), ('TTTATAT', 1946), ('TTATTTT', 1940), ('AAAAGAA', 1912), ('AATATAT', 1904), ('ATATATT', 1898), ('TTTTATT', 1889), ('TTTATTT', 1880), ('TTTTTCT', 1864), ('ATAAAAA', 1862), ('AAAAAAG', 1861), ('CTTTTTT', 1853), ('TATTATT', 1824), ('TAAAAAA', 1813), ('CAAAAAA', 1807), ('AGAAAAA', 1804), ('TATATTT', 1797), ('AAAATAT', 1781), ('TTTTTAT', 1780), ('AAATATA', 1771), ('TATATAA', 1734), ('AATAATA', 1729), (

2\. Quel sont les trois motifs de taille 7 les plus frequents? Pensez vous que ces motifs correspondent à de facteur de transcription connus? Justifier votre reponse.

<font color='blue'>
reponse: <br/>
les motifs les plus fréquentes ('AAAAAAA', 8385), ('TTTTTTT', 7658), ('ATATATA', 3096) pour le brin "+";<br/>
('TTTTTTT', 8385), ('AAAAAAA', 7658), ('TATATAT', 3096) pour le brin "-"<br/>
On pense qu'il est peu possible que c'est ces motifs correspondent à de facteur de transcription connus, car ils sont très réguliers (simple répétition des nucléotides).
</font>

3\. Le motif TGATTCAT correspond au facteur de transcription Gcn4 qui est trés suivant trouvé dans le genome de levures. Est-ce que vous avez trouvé ce motif? Si oui avec quel frequence?

In [8]:
motifGcn4 = ['TGATTCA']
def searchGivenMotif(motifsTrouve, motifSpecifique):
    """
    Cherche une liste de motifs specifiques dans un dictionaire de motifs trouvés
    entrée motifsTrouve : dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée motifSpecifique: liste de motifs specifiques à chercher
    sortie None : affiche le ranking des motifs especifique et sa frequence d'observation
    >>>searchGivenMotif({'TAA': 4, 'AAG': 3, 'AGT': 1}, ['AAG'])
    2 - AGG - 3
    """
    sortedMotifTrouver = sorted(motifsTrouve.items(), key=lambda item:item[1], reverse=True)
    sortedListMotifOnly = [motif for (motif, occurrence) in sortedMotifTrouver]
    for givenMotif in motifSpecifique:
        print("{} - {} - {}".format(sortedListMotifOnly.index(givenMotif)+1, givenMotif,motifsTrouve[givenMotif]))
    return None

motifsData = motifsP2
motifsRevComData = motifsReverseP2

searchGivenMotif(motifsData, motifGcn4)    
print ("reverse")
searchGivenMotif(motifsRevComData, motifGcn4)   

1819 - TGATTCA - 591
reverse
1791 - TGATTCA - 594


4\. Les motifs peu complexes (avec par exemple 5, 6 ou 7 fois la même lettre) sont courants dans les génomes, ils n'ont généralement pas de signification biologique. Dans le context de ce projet, vous pouvez eliminer aussi les motifis ayant deux lettre repetés, comme par exemple AGAGAGA. Faites une fonction pour éliminer les motifs peu complexes, _i. e._ qui ont au moins $m$ fois la même lettre ou qui ont deux lettre consecutive répétés. Dans quel position vous trouver Gcn4 apres enlever les motifs peu complexes?</font>. 

In [9]:
repeatMax = 5
simpleRepeatMax = 5

def isRepeatN_(seq, repeatMax):
    nonPermisList = [nuc*repeatMax for nuc in "ACGT"]
    for nonPermi in nonPermisList:
        if seq.find(nonPermi) != -1:
            return True
    return False

def isSimpleRepeat_(seq, simpleRepeatMax):
    if len(seq) <= 3:
        return False
    a = seq[0]
    b = seq[1]
    repeat = "".join([a if i%2 == 0 else b for i in range(simpleRepeatMax)])
    return seq.find(repeat) != -1

def isRepeatN(seq):
    return isRepeatN_(seq, repeatMax)
    
def isSimpleRepeat(seq):
    return isSimpleRepeat_(seq, simpleRepeatMax)

assert(isSimpleRepeat("AGAGAGAT"))
assert(isRepeatN("AAAAAA"))

In [10]:

def removeLowComplexe(motifs):
    """
    Eleve les motifs peu complexe 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie validMotif: dictionnaire de motifs filtré, clé=motif, valeur = fréquence d'observation
    """
    validMotif = dict()
    for (motif, freq) in motifs.items():
        if((not isSimpleRepeat(motif)) and (not isRepeatN(motif))):
            validMotif[motif] = freq
    return validMotif



motifsFiltreData = removeLowComplexe(motifsData)
motifsFiltreReverse = removeLowComplexe(motifsRevComData)
print(sortDictMotifToList(motifsFiltreData)[:top])
print(sortDictMotifToList(motifsFiltreReverse)[:top])

[('AAAGAAA', 2417), ('AAAATTT', 2308), ('AAATTTT', 2305), ('AAGAAAA', 2243), ('TTTATTT', 2164), ('TTTCTTT', 2159), ('TTATTTT', 2134), ('TTTTATT', 2110), ('AAAAGAA', 2079), ('TTTTCTT', 2052), ('AAAATAA', 1940), ('TTCTTTT', 1912), ('AATATAT', 1898), ('AATAAAA', 1889), ('AAATAAA', 1880), ('AATAATA', 1824), ('AAATATA', 1797), ('ATATTTT', 1781), ('TTATATA', 1734), ('TATTATT', 1729), ('AAAATAT', 1702), ('TTGAAAA', 1627), ('AATATAA', 1624), ('AAAACAA', 1600), ('AAGAAGA', 1599), ('ATAATAT', 1598), ('AAACAAA', 1598), ('AACAAAA', 1594), ('TAATATA', 1590), ('AATATTT', 1583), ('TTTGAAA', 1570), ('ATAAATA', 1564), ('AGAAGAA', 1557), ('ATTATTT', 1554), ('AAATATT', 1549), ('ATAATAA', 1545), ('ATTTATT', 1530), ('TATAATA', 1518), ('TTATATT', 1511), ('AAATAAT', 1508), ('ATTATAT', 1508), ('ATATTAT', 1505), ('TTATTAT', 1503), ('TTTCAAA', 1500), ('TTTTCAA', 1485), ('TATTATA', 1485), ('TATTTAT', 1483), ('TTTTGTT', 1478), ('TAATATT', 1465), ('ATTTTAT', 1448)]
[('TTTCTTT', 2417), ('AAATTTT', 2308), ('AAAATTT'

5\. Le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer les fonctions precedents pour chercher les 3 motifs les plus fréquents dans les deux brim. Il faut bien evidement enlever les motifs peu complexe.

In [13]:
k=8
feq=5
top= 30
reLow = 5

genome = "Data/ChipSeq/Sequence_by_Peaks_2.fasta"

sequence   = readFasta(genome)

print("Searching fwd motifs...\n")
fwdMotif = searchMotifs(k, sequence)

print("The most abundant motifs are")
topMotif = getTopMotifs(fwdMotif, top)
sortedTopMotif = sortDictMotifToList(topMotif)
print(sortedTopMotif)
print()

print("The most abundant reverse compl motifs are")
reverseSequence = [reversecompl(seq) for seq in sequence]
reverseFwdMotif = searchMotifs(k, reverseSequence)
reverseTopMotif = getTopMotifs(reverseFwdMotif, top)
sortedReverseTopMotif = sortDictMotifToList(reverseTopMotif)
print(sortedReverseTopMotif)
print()

print("The most abundant no low-complexe motifs are")
fwdMotifFiltrated = removeLowComplexe(fwdMotif)
fwdMotifFiltratedTop = getTopMotifs(fwdMotifFiltrated, top)
fwdMotifFiltratedTopSorted = sortDictMotifToList(fwdMotifFiltratedTop)
print(fwdMotifFiltratedTopSorted)
print()


print("The most abundant no low-complexe motifs in reverse compl are")
reverseFwdMotifFiltrated = removeLowComplexe(reverseFwdMotif)
reverseFwdMotifFiltratedTop = getTopMotifs(reverseFwdMotifFiltrated, top)
reverseFwdMotifFiltratedTopSorted = sortDictMotifToList(reverseFwdMotifFiltratedTop)
print(reverseFwdMotifFiltratedTopSorted)
print()


Searching fwd motifs...

The most abundant motifs are
[('AAAAAAAA', 36), ('TTATTATT', 17), ('GAGCAAAT', 14), ('TTTTTATT', 14), ('TTTATTAT', 13), ('ATTTGCTC', 12), ('TTTTTTTT', 12), ('ATATATAT', 12), ('TTTTATTA', 12), ('ATTATTAT', 11), ('TTATCATT', 11), ('TAGTTGTA', 10), ('ATTATTAG', 10), ('CAAAAAAA', 9), ('TTTGCTCA', 9), ('AGCAAATA', 9), ('GCAAATAT', 9), ('TTATTTTT', 9), ('AAAAGAAA', 9), ('AGAAAAAA', 9), ('ATTATTTT', 9), ('TATTATTT', 9), ('TATTATTA', 9), ('ATTATCAT', 9), ('ATTATTTG', 9), ('TTTATTTA', 9), ('TTATTTAT', 9), ('TTCTTCTC', 9), ('AAAAAAAT', 8), ('ATTTTTTT', 8)]

The most abundant reverse compl motifs are
[('TTTTTTTT', 36), ('AATAATAA', 17), ('ATTTGCTC', 14), ('AATAAAAA', 14), ('ATAATAAA', 13), ('GAGCAAAT', 12), ('AAAAAAAA', 12), ('ATATATAT', 12), ('TAATAAAA', 12), ('ATAATAAT', 11), ('AATGATAA', 11), ('TACAACTA', 10), ('CTAATAAT', 10), ('TTTTTTTG', 9), ('TGAGCAAA', 9), ('ATATTTGC', 9), ('TATTTGCT', 9), ('AAAAATAA', 9), ('TTTCTTTT', 9), ('TTTTTTCT', 9), ('AAAATAAT', 9), ('AAATA

6\. Ulitser la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ?


In [ ]:
Pour `Sequence_by_Peaks_2`, on pense que Hap4 est le facteur de transcription impliqué. Car la séquence 'ACCAATGA' de fréquence 10 et 'ACCAATCA' de fréquence 9 comprennent 'CCAAT', le site de fixation de Hap4. 